# Lec_19_ResNet

<font size=5><b><b></font>
<div align='right'> Hoe Sung Ryu ( 류 회 성 ) </div>
<div align='right'> Minsuk Sung ( 성 민 석) </div>
    
    

![](https://miro.medium.com/max/6652/1*OFfO8VzLv8GNFNRKafvB7w.png)
     
    
    
> Author: Hoe Sung Ryu, Minsuk Sung  <p>
> Tel: 010-6636-7275 / skainf23@gamil.com // 010-5134-3621 / mssung94@gmail.com  <p>
> 본 내용은 파이토치를 활용한 딥러닝 과외 자료입니다. 본 내용을 제작자의 동의없이 무단으로 복제하는 행위는 금합니다.
    

---

Syllabus
    
|Event Type|Date|Topic|
|--:|:---:|:---|
|1 |July 27| Environment setting and Python basic|
|2 |July 28| Pytorch basic and Custom Data load |
|3 |July 29| Traditional Machine Learning(1) |
|4 |July 30| Traditional Machine Learning(2) |
|5 |July 31| CNN(Convolutional Neural Network)(1)  |
|6 |Aug 03| CNN(Convolutional NeuralNetwork)(2) |
|7 |Aug 04|  RNN(Recurrent Neural Networks)(1) |
|8 |Aug 05|  RNN(Recurrent Neural Networks)(2) |
|9 |Aug 06|  Transfer learning(VGG pertained on ImageNEt for CIfar-10)| 
|10|Aug 07|**Mini_Kaggle**: Facial Expression Recognition on `AffectNet` | 
|11|Aug 08|`Awards` and `Closing`| 


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#ResNet" data-toc-modified-id="ResNet-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>ResNet</a></span><ul class="toc-item"><li><span><a href="#residual-block" data-toc-modified-id="residual-block-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>residual block</a></span></li></ul></li><li><span><a href="#Pretrained-models-in--PyTorch" data-toc-modified-id="Pretrained-models-in--PyTorch-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pretrained models in  PyTorch</a></span></li></ul></div>

## ResNet

![](https://miro.medium.com/max/6652/1*OFfO8VzLv8GNFNRKafvB7w.png)


References
- [1] He, K., Zhang, X., Ren, S., & Sun, J. (2016). Deep residual learning for image recognition. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 770-778). ([CVPR Link](https://www.cv-foundation.org/openaccess/content_cvpr_2016/html/He_Deep_Residual_Learning_CVPR_2016_paper.html))

- [2] Zhang, K., Tan, L., Li, Z., & Qiao, Y. (2016). Gender and smile classification using deep convolutional neural networks. In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition Workshops (pp. 34-38).

### residual block
The ResNet-50 architecture is similar to the ResNet-34 architecture shown below (from [1]):
<img src='../img/resnet-50-bottleneck.png'>

However, in ResNet-50, the skip connection uses a bottleneck (from [1]):

![](../img/resnet50-arch-1.png)


---

## Pretrained models in  PyTorch

Pretrained models
- AlexNet
- VGG
- ResNet
- SqueezeNet
- DenseNet
- Inception v3
- GoogLeNet
- ShuffleNet v2
- MobileNet v2
- ResNeXt
- Wide ResNet
- MNASNet

```python
import torchvision.models as models
resnet18 = models.resnet18()
alexnet = models.alexnet()
vgg16 = models.vgg16()
squeezenet = models.squeezenet1_0()
densenet = models.densenet161()
inception = models.inception_v3()
googlenet = models.googlenet()
shufflenet = models.shufflenet_v2_x1_0()
mobilenet = models.mobilenet_v2()
resnext50_32x4d = models.resnext50_32x4d()
wide_resnet50_2 = models.wide_resnet50_2()
mnasnet = models.mnasnet1_0()
```

reference: [PyTorch TORCHVISON.MODELS](https://pytorch.org/docs/stable/torchvision/models.html)

<div class="alert alert-success" data-title="">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise :  ResNet_50
  </h1>
</div>

In [9]:
import torch
import torch.nn as nn


class block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        """
        identity_downsample = dimension
        """
        
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels,
                               out_channels,
                               kernel_size=1,
                               stride=1,
                               padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels) # bn
        self.conv1 = nn.Conv2d(out_channels,
                               out_channels,
                               kernel_size=3,
                               stride=stride,
                               padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv1 = nn.Conv2d(out_channels,
                               out_channels*self.expansion,
                               kernel_size=1,
                               stride=1,
                               padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

        def forward(self, x):
            identity = x
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.conv2(x)
            x = self.bn2(x)
            x = self.conv3(x)
            x = self.bn3(x)

            # Residual reshape !!!
            if self.identity_downsample is not None:
                identity = self.identity_downsample(identity)
            x += identity

            x = self.relu(x)


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # layer1_block
        self.layer1 = self._maker_layer(block, layers[0], out_channels=64, stride=1)
        # layer2_block
        self.layer2 = self._maker_layer(block, layers[1], out_channels=128, stride=1)
        # layer3_block
        self.layer3 = self._maker_layer(block, layers[2], out_channels=256, stride=1)
        # layer4_block
        self.layer4 = self._maker_layer(block, layers[3], out_channels=512, stride=1)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*4, num_classes)
        
    def forward(self,x):
        x = self.conv1(X)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # layer 
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = self.fc(x)
        return x 
        

    def _maker_layer(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []

        if (stride != 1) or (self.in_channels != out_channels * 4):
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, 
                                                          out_channels*4,
                                                          kernel_size=1,
                                                          stride=stride),
                                                nn.BatchNorm2d(out_channels*4)
                                               )
        # 1st residual block 
        layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
        
        # add rest residual block 
        for i in range(num_residual_blocks -1):
            layers.append(block(self.in_channels, out_channels))
            
        return nn.Sequential(*layers)                        
    
    
    
def ResNet50(img_channels, num_classess):
    return ResNet(block,[3,4,6,3], img_channels, num_classess)

def ResNet101(img_channels, num_classess):
    return ResNet(block,[3,4,23,3], img_channels, num_classess)

def ResNet152(img_channels, num_classess):
    return ResNet(block,[3,8,36,3], img_channels, num_classess)

In [10]:
model = ResNet50(1, 10)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  
import torch.nn as nn 

criterion = nn.CrossEntropyLoss()

NameError: name 'LEARNING_RATE' is not defined

In [ ]:
def compute_accuracy(model, data_loader):
    model.eval()
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)

#         logits, probas = model(features)

        predicted= model(features)
        _, predicted_labels = torch.max(predicted, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100


def compute_epoch_loss(model, data_loader):
    model.eval()
    curr_loss, num_examples = 0., 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.to(DEVICE)
            targets = targets.to(DEVICE)
#             logits, probas = model(features)
            predicted= model(features)
#             loss = F.cross_entropy(logits, targets, reduction='sum')
            loss = criterion(predicted,targets,reduction='sum' )
            num_examples += targets.size(0)
            curr_loss += loss

        curr_loss = curr_loss / num_examples
        return curr_loss
    
    

In [ ]:
start_time = time.time()
for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
#         print(features.shape)
#         break
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
#         logits, probas = model(features)
#         cost = F.cross_entropy(logits, targets)

        predicted = model(features)
        loss = criterion(predicted,targets)
        optimizer.zero_grad()
        loss.backward()
#         cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%% |  Loss: %.3f' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader),
              compute_epoch_loss(model, train_loader)))


    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))